# Import dependencies

In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp

# Capture keypoints

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return results, image

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(200,0,50), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(160,0,0), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))

# Extract Keypoints

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

# Folder Setup

In [6]:
actions = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S',
                    'T','U','V','W','X','Y','Z','HELLO','HEY','_I','MY','NAME','YOU','YOUR','HOW ARE YOU',
                    'I\'M FINE','NICE','TO MEET YOU','WHERE','FROM','LIKE','WORK','WORK','MOVIE','WHAT',
                    'WHAT TIME IS IT','WASHROOM','MEANING','SORRY','BAD','LOVE','WITH','WANT TO',
                    'FAVOURITE','SIGN','PLEASE','THANK YOU','SEE YOU LATER','GOOD','MORNING','AFTERNOON',
                    'NIGHT','TODAY','GO','COME','EXCUSE ME','GOOD BYE','TAKE CARE','FOR WATCHING','HAVE',
                    'DAY','YES','NO','KNOW','LITTLE','EVERYONE','0','1','2','3','4','5','6','7','8','9'])

"""actions = np.array(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S',
                    'T','U','V','W','X','Y','Z'])     ## CHANGE THE ARRAY ACCORDING TO YOUR NEED 
                        ##  **(FIRST TIME EXECUTE WITH ALL PHRASES YOU WILL WORK FOR -- TO FOLDER CREATION)"""

DATA_PATH = os.path.join('DATA')
no_of_videos = 31
no_of_frames = 30

In [7]:
## ONLY ONE TIME EXECUTION IS NEEDED
def setup_folder(actions):
    for action in actions:
        for video_no in range(-1,no_of_videos):
            try:
                os.makedirs(os.path.join(DATA_PATH,action,str(video_no)))
            except:
                print("Cannot create directorie!")

In [8]:
## ONLY ONE TIME EXECUTION IS NEEDED
setup_folder(actions)

# Data collection

In [9]:
def data_collect(actions):
    no_of_videos = 31
    no_of_frames = 30
    camera = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
        for action in actions:
            for video_no in range(-1,no_of_videos):
                for frame_no in range(no_of_frames):
                    success, frame = camera.read()

                    results, image = mediapipe_detection(frame,holistic)

                    draw_landmarks(image, results)
                    
                    if frame_no == 0:
                        cv2.putText(image, 'Sart Recording...',(250,250),cv2.FONT_HERSHEY_SIMPLEX,
                                   1,(0,255,0),3,cv2.LINE_AA)
                        cv2.putText(image,'Data for:{} video_no:{}'.format(action,video_no),
                                    (15,60),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2,cv2.LINE_AA)
                        
                        cv2.imshow('Data Collection...',image)
                        
                        cv2.waitKey(1000)
                    else:
                        cv2.putText(image,'Data for:{} video_no:{}'.format(action,video_no),
                                    (15,60),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2,cv2.LINE_AA)
                        cv2.imshow('Data Collection...',image)
                    
                    keypoint_array = extract_keypoints(results)
                    
                    array_path = os.path.join(DATA_PATH,action,str(video_no),str(frame_no))
                    
                    np.save(array_path,keypoint_array)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
        camera.release()
        cv2.destroyAllWindows()

In [10]:
data_collect(actions)   ## EXECUTE TO START DATA COLLECTION (VIDEO RECORDING)

In [1]:
model.load_weights('model.h5')

NameError: name 'model' is not defined